In [14]:
using Plots
pyplot()

Plots.PyPlotBackend()

HermiteGf is a julia package, install it with

```julia
using Pkg
Pkg.add("https://gitlab.mpcdf.mpg.de/clapp/HermiteGF.jl")
```

In [15]:
using HermiteGF

In [16]:
" Node positions (Uniform or Chebyshev) "
abstract type NodesType end

NodesType

In [17]:
import HermiteGF:generate_chebyshev_nodes

In [18]:
"""
    Chebyshev( nx, xmin, xmax )

Chebyshev nodes

"""
struct Chebyshev <: NodesType

    nx    :: Int64
    xmin  :: Float64
    xmax  :: Float64
    xk    :: Vector{Float64}

    function Chebyshev( nx, xmin, xmax )

        xk  = zeros(Float64, nx)
        xk .= generate_chebyshev_nodes(nx, xmin, xmax)
        new( nx, xmin, xmax, xk )

    end

end

Chebyshev

In [19]:
import HermiteGF:generate_uniform_nodes

In [20]:
"""
    Uniform( nx, xmin, xmax )

Uniform nodes

"""
struct Uniform <: NodesType

    nx    :: Int64
    xmin  :: Float64
    xmax  :: Float64
    xk    :: Vector{Float64}

    function Uniform( nx, xmin, xmax )

        xk  = zeros(Float64, nx)
        xk .= generate_uniform_nodes(nx, xmin, xmax)
        new( nx, xmin, xmax, xk )

    end

end

Uniform

In [21]:
abstract type InterpolationType end

In [22]:
import HermiteGF:evaluate_hermite

In [23]:
"""
    Hermite( nodes::NodesType, epsilon, gamma )

Hermite inteprolation

"""
mutable struct Hermite <: InterpolationType

    nx         :: Int64
    dims       :: Int64
    epsilon    :: Float64
    gamma      :: Float64
    xk         :: Array{Float64,1}
    colloc_mat :: Array{Float64,2}

    function Hermite( nodes::NodesType, epsilon, gamma )

        dims = 1
        xk   = nodes.xk
        nx   = nodes.nx
        colloc_mat = evaluate_hermite(xk, nx, epsilon, gamma)
        new( nx, dims, epsilon, gamma, xk, colloc_mat )

    end

end

Hermite

In [24]:
"""
    Hermite( xe )

returns evaluation matrix
"""
function (interp::Hermite)( xe::Array{Float64,1} )

   nx      = interp.nx
   epsilon = interp.epsilon
   gamma   = interp.gamma
   evaluate_hermite(xe, nx, epsilon, gamma)

end

Hermite

In [25]:
import HermiteGF:evaluate_radial

In [26]:
"""
    Radial( nodes::NodesType, epsilon, gamma )

Radial inteprolation

"""
mutable struct Radial <: InterpolationType

    nx         :: Int64
    epsilon    :: Float64
    gamma      :: Float64
    xk         :: Array{Float64,1}
    colloc_mat :: Array{Float64,2}

    function Radial( nodes::NodesType, epsilon, gamma )
        
        dims       = 1
        xk         = nodes.xk
        nx         = nodes.nx
        colloc_mat = evaluate_radial(xk, xk, nx, epsilon, gamma)
        new( dims, epsilon, gamma, xk, colloc_mat )
        
    end

end

Radial

In [27]:
"""
    Radial( xe )

returns evaluation matrix
"""
function (interp::Radial)( xe::Array{Float64,1} )

    nx      = interp.nx
    epsilon = interp.epsilon
    gamma   = interp.gamma
    xk      = interp.xk
    evaluate_radial(xk, xe, nx, epsilon, gamma)

end

Radial

In [28]:
import HermiteGF:evaluate_s_1D

In [13]:
function interpolate(interp::InterpolationType, 
        f::Array{Float64,1}, xe::Array{Float64,1} )

   ϵ  = interp.epsilon
   nx = interp.nx
   xk = interp.xk

   # Precompute 1D interpolation matrix
   x_all = interp(xe) / interp.colloc_mat

   # Compute the interpolated values
   evaluate_s(x_all, f)

end


# interpolation domain.
xmin, xmax, nx = -1, 1, 32

ϵ = 0.1
γ = 3

hermite = Hermite(Chebyshev(nx, xmin, xmax), ϵ, γ)
radial  = Radial(Chebyshev(nx, xmin, xmax), ϵ, γ)
xk = interpolation.xk
f  =  cos.(xk.^2)
xe = collect(range(xmin, stop=xmax, length=nx))
plot(xe, interpolate( hermite, f, xe ))
plot(xe, interpolate( radial, f, xe ))
scatter!(xk, f)

UndefVarError: UndefVarError: Chebyshev not defined

In [29]:
using IterTools

In [33]:
for k, v in product([:Hermite, :Radial],[:Uniform, :Chebyshev])
    println(k, v)
end

LoadError: syntax: invalid iteration specification

In [21]:
#l2_error = sqrt(trapz_1D((s - f_vals).^2, dx))
#l1_error = maximum(abs.(s .- f_vals))
#
#f_vals = h(xx_e)

32-element Array{Float64,1}:
 0.54030230586814  
 0.6408970161773455
 0.7258102410433283
 0.7958633344738472
 0.8522796739163705
 0.8965416349514189
 0.9302692145972262
 0.9551209764487195
 0.9727162883451472
 0.9845767075724988
 0.99208373838787  
 0.9964499363706495
 0.9987003653073915
 ⋮                 
 0.9964499363706478
 0.9920837383878681
 0.984576707572498 
 0.9727162883451476
 0.95512097644872  
 0.9302692145972272
 0.896541634951419 
 0.8522796739163696
 0.7958633344738454
 0.7258102410433296
 0.6408970161773515
 0.540302305868137 